In [3]:
import sys
sys.path.append('..')
#WORKS ON TORCH==1.4.1, schnetpack == 0.3
from aimnet import load_AIMNetMT_ens, load_AIMNetSMD_ens
import ase
import ase.optimize
import ase.md
import numpy as np

### Load pre-trained esnsembles of AIMNet models

load_AIMNetMT_ens - loads a model trained to wB97x/def2-TZVPP energies, atomic electric moments (charges, dipoles, etc) and volumes

load_AIMNetSMD_ens - loads a model obtained by transfer learning towards SMD-wB97x/def2-TZVPP energies

In [2]:
import torch
import schnetpack as spk
from schnetpack.datasets import QM9

qm9_filepath = '../../data/datasets/QM9/qm9.db'
qm9_data = QM9(qm9_filepath,download=False,remove_uncharacterized=True)

afv_embs = np.zeros((1,16))
aef_embs = np.zeros((1,256))
for each_molecule in range(10000):

    if each_molecule % 1000 == 0:
        print(each_molecule)

    atoms, props = qm9_data.get_properties(each_molecule)
    converter = spk.data.AtomsConverter(device='cpu')
    inputs = converter(atoms)


    atomic_numbers_to_idx = {1:  0,
                            6:  1,
                            7:  2,
                            8:  3,
                            16: 4,
                            9:  5,
                            17: 6}

    atomic_numbers = inputs['_atomic_numbers'].detach().numpy()

    atomic_numbers_in_idx = [atomic_numbers_to_idx[atomic_numbers[0][each_element]] for each_element in range(len(inputs['_atomic_numbers'][0]))]

    atomic_numbers_in_idx = torch.tensor([atomic_numbers_in_idx])


    #use model on inputs  
    model_gas = load_AIMNetMT_ens()
    pred, afv, aef = model_gas(inputs['_positions'],atomic_numbers_in_idx)

    afv = afv.detach().numpy()
    aef = aef.detach().numpy()

    afv_embs = np.vstack((afv_embs,afv[0]))
    aef_embs = np.vstack((aef_embs,aef[0]))



0


c:\Users\aelsamma\.conda\envs\aimnet2\lib\site-packages\ase\atoms.py:968: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)


KeyboardInterrupt: 

In [3]:
np.savetxt('../../afv.csv',afv_embs,delimiter=',')
np.savetxt('../../aef.csv',aef_embs,delimiter=',')

#### Get atomic charges and volumes

In [ ]:
atoms.set_calculator(calc_smd)

print(calc_smd.results)



In [ ]:
charges = calc_smd.results['elmoments'][0, :, 0]
print('Charges: ', charges)